In [11]:
import json
from urllib.request import urlopen #내장 library인 urllib에서 reqeuest 함수를 호출한다.
from bs4 import BeautifulSoup

In [4]:

jsonString = """{"arrayOfNums":[{"number":0}, {"number":1},{"number":2}],
                "arrayOfFruits":[{"fruit":"apple"}, {"fruit":"banana"}, {"fruit": "pear"}]
                }"""

jsonObj = json.loads(jsonString) #jsonString이란 json형식을 load하여 jsonObj에 저장. 이때 타입은 dict이다.

#dict 문법으로 내부 데이터를 볼 수 있다.
print(jsonObj.get('arrayOfNums'))  
print(jsonObj.get('arrayOfNums')[1])
print(jsonObj.get('arrayOfNums')[1].get('number') + jsonObj.get('arrayOfNums')[2].get('number'))
print(jsonObj.get('arrayOfFruits')[2].get('fruit'))


[{'number': 0}, {'number': 1}, {'number': 2}]
{'number': 1}
3
pear


In [6]:
type(jsonObj)

dict

In [7]:
from urllib.request import urlopen #내장 library인 urllib에서 reqeuest 함수를 호출한다.
html = urlopen('http://pythonscraping.com/pages/page1.html')  #열 url을 입력한다.
print(html.read()) #read로 html 태그를 모두 읽어온다.

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('http://pythonscraping.com/pages/page1.html')
bs = BeautifulSoup(html.read(), 'html.parser') #2개의 매개변수. [1: html텍스트, 2: 구문 분석기(주로 html.parser를 사용한다. lxml도 있다. lxml의 경우 지저분한 html을 대상으로 효과적이다.)]
#html.parser 외의 구문 분석기 
#xlml
#html5lib

print(bs)

<html>
<head>
<title>A Useful Page</title>
</head>
<body>
<h1>An Interesting Title</h1>
<div>
Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>
</body>
</html>



In [15]:
bs.div

<div>
Lorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.
</div>

# 신뢰할 수 있는 연결과 예외 처리

스크래핑을 하면 생각지 못한 부분에서 에러가 발생한다. 이는 홈페이지를 잘 못 만든 사람의 탓이 아니라 예상하지 못한 사용자 탓이다.  
이런 경우를 해결하기 위한 예외처리를 공부하자.

In [13]:
html = urlopen('http://www.pythonscraping.com/pages/page1.html')
bs = BeautifulSoup(html.read(), 'html.parser')

In [6]:
html = urlopen('http://www.pythonscraping.com/pages/error.html')
html

HTTPError: HTTP Error 404: Not Found

이 행에서 발생할 수 있는 에러는 두 가지다.  
1. 페이지를 찾을 수 없고, URL해석에 에러가 생긴 경우   
    -> 404 Page Not Found, 500 Internal Server Error  
    -> 범용 예외인 HTTPError를 일으킴
    
2. 서버를 찾을 수 없는 경우  


In [10]:
# HTTPError 예외처리

from urllib.request import urlopen
from urllib.request import HTTPError

try: 
    html = urlopen('http://www.pythonscraping.com/pages/error.html')
#     html = urlopen('http://www.pythonscraping.com/pages/page1.html')
except HTTPError as e:
    print(e)
except URLError as e:
    print('The server could not be found!')
else:
    print('It Worked!')


HTTP Error 404: Not Found


서버가 존재한다하더라도, 객체에 들어있는 태그가 존재하는지에 따라 오류가 발생할 수 있다.

In [17]:
#None 객채를 반환함
bs.nonExistentTag

In [18]:
#없는 태그에 접근을 시도하면 AttributeError가 발생함
bs.nonExistentTag.someTag

AttributeError: 'NoneType' object has no attribute 'someTag'

In [20]:
#위 두 상황을 대체하는 방법
try:
    badContent = bs.find("nonExistingTag").anotherTag #'.nonExistingTag'는 '.find("nonExistingTag")'로 바뀌었다.
except AttributeError as e:
    print('Tag was not found')
else:
    if badContent == None:
        print('Tag was not found')
    else:
        print(badContent)

Tag was not found


### 재사용 가능성을 높인 예외처리 함수 예시

이 함수는 페이지 타이틀을 반환하거나, 문제가 있으면 None 객체를 반환한다.

In [22]:
def getTitle(url):
    try:
        html = urlopen(url)
    except HTTPError as e:
        return None
    
    try:
        bs = BeautifulSoup(html.read(), 'html.parser')
        title = bs.body.h1
    except AttributeError as e:
        return None
    
    return title

title = getTitle('http://www.pythonscraping.com/pages/page1.html')
# title = getTitle('http://www.pythonscraping.com/pages/error.html')
if title ==None:
    print('Title could not be found')
else:
    print(title)

Title could not be found


In [ ]:
import 